# Kapitola 30: Aktivacni funkce - Dejte neuronu osobnost

## Cile kapitoly
- Pochopit roli aktivacnich funkci v neuronove siti
- Naucit se implementovat Sigmoid, Tanh a ReLU
- Vizualizovat a porovnat vlastnosti ruznych aktivacnich funkci
- Pochopit problem mizejiciho gradientu (vanishing gradient)
- Porovnat vliv aktivacnich funkci na uceni site

## Predpoklady
- Zaklady prace s NumPy a Matplotlib
- Pochopeni Perceptronu a vicevrstvych siti (kapitoly 28-29)

## 1. Co je aktivacni funkce?

V minulych kapitolach jsme videli, ze neuron pocita vazeny soucet vstupu:

$$z = w_1 \cdot x_1 + w_2 \cdot x_2 + ... + w_n \cdot x_n + b$$

Ale co s timto cislem udelame? **Aktivacni funkce** rozhoduje, jak silny signal neuron posle dal.

### Proc potrebujeme aktivacni funkce?

1. **Nelinearita** - Bez aktivacni funkce by cela sit byla jen linearni transformace
2. **Omezeni vystupu** - Nekteré funkce omezi vystup do urciteho rozsahu (0-1, -1 az 1)
3. **Rozhodovani** - Neuron se musi "rozhodnout", jak moc ma byt aktivni

In [ ]:
# Instalace potrebnych knihoven
!pip install numpy matplotlib scikit-learn torch -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("Knihovny uspesne nacteny!")

## 2. Skokova funkce (Step Function)

Nejjednodussi aktivacni funkce - pouzival ji puvodni Perceptron.

$$f(x) = \begin{cases} 1 & \text{pokud } x \geq 0 \\ 0 & \text{jinak} \end{cases}$$

**Vyhody:** Jednoducha, jasne rozhodnuti (0 nebo 1)

**Nevyhody:** Neni diferencovatelna (nelze pouzit gradient descent)

In [ ]:
# Implementace skokove funkce
def step_function(x):
    """Skokova funkce - 0 nebo 1"""
    return np.where(x >= 0, 1, 0)

# Vizualizace
x = np.linspace(-5, 5, 200)

plt.figure(figsize=(10, 5))
plt.plot(x, step_function(x), 'b-', linewidth=2, label='Step Function')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Vstup (z)', fontsize=12)
plt.ylabel('Vystup f(z)', fontsize=12)
plt.title('Skokova funkce (Step Function)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(-0.2, 1.2)
plt.show()

print("Priklady:")
print(f"step(-2) = {step_function(-2)}")
print(f"step(0) = {step_function(0)}")
print(f"step(3) = {step_function(3)}")

## 3. Sigmoid funkce

Klasicka "S-krivka", ktera zmackne jakykoliv vstup do rozsahu (0, 1).

$$\sigma(x) = \frac{1}{1 + e^{-x}}$$

**Vyhody:**
- Hladka a diferencovatelna (lze pouzit gradient descent)
- Vystup lze interpretovat jako pravdepodobnost

**Nevyhody:**
- Problem mizejiciho gradientu pri velmi velkych/malych hodnotach
- Vystup neni centrovany kolem nuly

In [ ]:
# Implementace Sigmoid
def sigmoid(x):
    """Sigmoid aktivacni funkce"""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Derivace sigmoidu - potrebna pro backpropagation"""
    s = sigmoid(x)
    return s * (1 - s)

# Vizualizace
x = np.linspace(-10, 10, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf funkce
axes[0].plot(x, sigmoid(x), 'b-', linewidth=2, label='Sigmoid')
axes[0].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='y = 0.5')
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Vstup (z)', fontsize=12)
axes[0].set_ylabel('Vystup sigma(z)', fontsize=12)
axes[0].set_title('Sigmoid funkce', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-0.1, 1.1)

# Graf derivace
axes[1].plot(x, sigmoid_derivative(x), 'g-', linewidth=2, label='Derivace Sigmoid')
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Vstup (z)', fontsize=12)
axes[1].set_ylabel('sigma\'(z)', fontsize=12)
axes[1].set_title('Derivace Sigmoid (max = 0.25)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Priklady hodnot Sigmoid:")
print(f"sigmoid(-5) = {sigmoid(-5):.4f} (blizko 0)")
print(f"sigmoid(0) = {sigmoid(0):.4f} (presne 0.5)")
print(f"sigmoid(5) = {sigmoid(5):.4f} (blizko 1)")

## 4. Tanh funkce (Hyperbolicka tangenta)

Podobna Sigmoidu, ale vystup je v rozsahu (-1, 1).

$$\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$

**Vyhody:**
- Vystup je centrovany kolem nuly (casto lepsi pro skryte vrstvy)
- Silnejsi gradienty nez Sigmoid

**Nevyhody:**
- Stale trpi problemem mizejiciho gradientu

In [ ]:
# Implementace Tanh
def tanh(x):
    """Tanh aktivacni funkce"""
    return np.tanh(x)

def tanh_derivative(x):
    """Derivace tanh"""
    return 1 - np.tanh(x)**2

# Vizualizace
x = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf funkce
axes[0].plot(x, tanh(x), 'orange', linewidth=2, label='Tanh')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Vstup (z)', fontsize=12)
axes[0].set_ylabel('Vystup tanh(z)', fontsize=12)
axes[0].set_title('Tanh funkce', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-1.2, 1.2)

# Graf derivace
axes[1].plot(x, tanh_derivative(x), 'red', linewidth=2, label='Derivace Tanh')
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Vstup (z)', fontsize=12)
axes[1].set_ylabel('tanh\'(z)', fontsize=12)
axes[1].set_title('Derivace Tanh (max = 1.0)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Priklady hodnot Tanh:")
print(f"tanh(-3) = {tanh(-3):.4f} (blizko -1)")
print(f"tanh(0) = {tanh(0):.4f} (presne 0)")
print(f"tanh(3) = {tanh(3):.4f} (blizko 1)")

## 5. ReLU (Rectified Linear Unit)

**Hvezda modernich neuronovych siti!** Jednoducha a efektivni.

$$\text{ReLU}(x) = \max(0, x)$$

**Vyhody:**
- Vypocetne velmi rychla
- Netrpi problemem mizejiciho gradientu (pro kladne hodnoty)
- V praxi funguje nejlepe pro vetsinu uloh

**Nevyhody:**
- Problem "umirajicich neuronu" (dying ReLU) - neuron muze "odumrit" a uz se neuci

In [ ]:
# Implementace ReLU
def relu(x):
    """ReLU aktivacni funkce"""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivace ReLU"""
    return np.where(x > 0, 1, 0)

# Vizualizace
x = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf funkce
axes[0].plot(x, relu(x), 'green', linewidth=2, label='ReLU')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Vstup (z)', fontsize=12)
axes[0].set_ylabel('Vystup ReLU(z)', fontsize=12)
axes[0].set_title('ReLU funkce', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graf derivace
axes[1].plot(x, relu_derivative(x), 'darkgreen', linewidth=2, label='Derivace ReLU')
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Vstup (z)', fontsize=12)
axes[1].set_ylabel('ReLU\'(z)', fontsize=12)
axes[1].set_title('Derivace ReLU (0 nebo 1)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-0.2, 1.5)

plt.tight_layout()
plt.show()

print("Priklady hodnot ReLU:")
print(f"relu(-3) = {relu(-3)} (negativa -> 0)")
print(f"relu(0) = {relu(0)}")
print(f"relu(3) = {relu(3)} (pozitiva -> beze zmeny)")

## 6. Leaky ReLU - Reseni problemu umirajicich neuronu

Varianta ReLU, ktera propusti male zaporne hodnoty.

$$\text{LeakyReLU}(x) = \begin{cases} x & \text{pokud } x > 0 \\ \alpha x & \text{jinak} \end{cases}$$

Kde $\alpha$ je typicky 0.01.

In [ ]:
# Implementace Leaky ReLU
def leaky_relu(x, alpha=0.01):
    """Leaky ReLU aktivacni funkce"""
    return np.where(x > 0, x, alpha * x)

# Vizualizace porovnani ReLU a Leaky ReLU
x = np.linspace(-5, 5, 200)

plt.figure(figsize=(10, 5))
plt.plot(x, relu(x), 'green', linewidth=2, label='ReLU')
plt.plot(x, leaky_relu(x, alpha=0.1), 'purple', linewidth=2, label='Leaky ReLU (alpha=0.1)')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Vstup (z)', fontsize=12)
plt.ylabel('Vystup', fontsize=12)
plt.title('ReLU vs Leaky ReLU', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Leaky ReLU propusti male zaporne hodnoty:")
print(f"leaky_relu(-5, alpha=0.1) = {leaky_relu(-5, 0.1)}")

## 7. Porovnani vsech aktivacnich funkci

Pojdme si vizualizovat vsechny funkce na jednom grafu.

In [ ]:
# Porovnani vsech aktivacnich funkci
x = np.linspace(-5, 5, 200)

plt.figure(figsize=(12, 7))

plt.plot(x, sigmoid(x), 'b-', linewidth=2, label='Sigmoid (0 az 1)')
plt.plot(x, tanh(x), 'orange', linewidth=2, label='Tanh (-1 az 1)')
plt.plot(x, relu(x), 'green', linewidth=2, label='ReLU (0 nebo x)')
plt.plot(x, leaky_relu(x, 0.1), 'purple', linewidth=2, linestyle='--', label='Leaky ReLU')

plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)

plt.xlabel('Vstup neuronu (vazeny soucet)', fontsize=12)
plt.ylabel('Vystup neuronu (aktivace)', fontsize=12)
plt.title('Porovnani aktivacnich funkci', fontsize=14)
plt.legend(loc='upper left', fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(-1.5, 5)
plt.xlim(-5, 5)
plt.show()

In [ ]:
# Porovnani derivaci (dulezite pro uceni!)
x = np.linspace(-5, 5, 200)

plt.figure(figsize=(12, 7))

plt.plot(x, sigmoid_derivative(x), 'b-', linewidth=2, label='Sigmoid\' (max 0.25)')
plt.plot(x, tanh_derivative(x), 'orange', linewidth=2, label='Tanh\' (max 1.0)')
plt.plot(x, relu_derivative(x), 'green', linewidth=2, label='ReLU\' (0 nebo 1)')

plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)

plt.xlabel('Vstup', fontsize=12)
plt.ylabel('Derivace', fontsize=12)
plt.title('Derivace aktivacnich funkci (klicove pro backpropagation)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.2)
plt.show()

print("\nProc jsou derivace dulezite?")
print("- Pri backpropagation se gradient nasobi derivaci")
print("- Sigmoid ma max derivaci 0.25 -> gradient rychle mizi")
print("- ReLU ma derivaci 1 pro kladne hodnoty -> gradient se neztraci")

## 8. Problem mizejiciho gradientu (Vanishing Gradient)

V hlubokych sitich se gradient nasobi pri kazde vrstve. Pokud je derivace aktivacni funkce mala, gradient rychle konverguje k nule.

Pojdme si to demonstrovat!

In [ ]:
# Simulace mizejiciho gradientu
def simulate_gradient_flow(activation_derivative, n_layers=10, initial_gradient=1.0):
    """Simuluje tok gradientu skrz vrstvy site"""
    gradients = [initial_gradient]
    current_gradient = initial_gradient
    
    # Predpokladame, ze aktivace jsou v typickem rozsahu kolem 0
    for layer in range(n_layers):
        # Gradient se nasobi derivaci v kazde vrstve
        derivative = activation_derivative(0)  # derivace v bode 0
        current_gradient = current_gradient * derivative
        gradients.append(current_gradient)
    
    return gradients

# Porovnani gradientu pro ruzne aktivacni funkce
layers = list(range(11))

sigmoid_gradients = simulate_gradient_flow(sigmoid_derivative)
tanh_gradients = simulate_gradient_flow(tanh_derivative)
relu_gradients = simulate_gradient_flow(relu_derivative)

plt.figure(figsize=(12, 6))

plt.plot(layers, sigmoid_gradients, 'b-o', linewidth=2, label='Sigmoid', markersize=8)
plt.plot(layers, tanh_gradients, 'orange', linewidth=2, marker='s', label='Tanh', markersize=8)
plt.plot(layers, relu_gradients, 'g-^', linewidth=2, label='ReLU', markersize=8)

plt.xlabel('Vrstva (od vystupu k vstupu)', fontsize=12)
plt.ylabel('Velikost gradientu', fontsize=12)
plt.title('Tok gradientu skrz 10 vrstev neuronove site', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Logaritmicka skala pro lepsi vizualizaci
plt.show()

print("\nGradient po 10 vrstvach:")
print(f"Sigmoid: {sigmoid_gradients[-1]:.10f} (prakticky nulovy!)")
print(f"Tanh:    {tanh_gradients[-1]:.10f}")
print(f"ReLU:    {relu_gradients[-1]:.10f} (zachovany!)")

## 9. Prakticke cviceni: Vliv aktivacni funkce na uceni

Pouzijeme sklearn MLPClassifier k porovnani, jak ruzne aktivacni funkce ovlivnuji uceni na datasetu "dva mesice".

In [ ]:
from sklearn.datasets import make_moons
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Vytvoreni datasetu
X, y = make_moons(n_samples=500, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vizualizace dat
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='black', alpha=0.7)
plt.xlabel('X1', fontsize=12)
plt.ylabel('X2', fontsize=12)
plt.title('Dataset "Dva mesice" - nelinearni problem', fontsize=14)
plt.colorbar(label='Trida')
plt.show()

In [ ]:
# Porovnani aktivacnich funkci
activations = ['identity', 'logistic', 'tanh', 'relu']
activation_names = ['Bez aktivace (linearni)', 'Sigmoid (logistic)', 'Tanh', 'ReLU']

results = {}

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for idx, (activation, name) in enumerate(zip(activations, activation_names)):
    # Trenovani modelu
    model = MLPClassifier(
        hidden_layer_sizes=(16,),
        activation=activation,
        max_iter=1000,
        random_state=42,
        solver='adam',
        learning_rate_init=0.01
    )
    model.fit(X_train, y_train)
    
    # Predikce a presnost
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    
    # Vizualizace rozhodovaci hranice
    ax = axes[idx]
    
    # Vytvoreni mrizky pro vizualizaci
    xx, yy = np.meshgrid(
        np.linspace(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5, 200),
        np.linspace(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5, 200)
    )
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='coolwarm', edgecolors='black')
    ax.set_title(f'{name}\nPresnost: {accuracy:.1%}', fontsize=12)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')

plt.tight_layout()
plt.show()

# Shrnuti vysledku
print("\n" + "="*50)
print("POROVNANI AKTIVACNICH FUNKCI")
print("="*50)
for name, acc in results.items():
    print(f"{name:30s}: {acc:.2%}")

## 10. Mini-projekt: Analyza rychlosti uceni

Porovname, jak rychle se site s ruznymi aktivacemi uci.

In [ ]:
# Analyza rychlosti uceni
activations_to_test = ['logistic', 'tanh', 'relu']
colors = {'logistic': 'blue', 'tanh': 'orange', 'relu': 'green'}

plt.figure(figsize=(12, 6))

for activation in activations_to_test:
    model = MLPClassifier(
        hidden_layer_sizes=(16,),
        activation=activation,
        max_iter=500,
        random_state=42,
        solver='sgd',
        learning_rate_init=0.1
    )
    
    # Trenovani s monitorovanim loss
    model.fit(X_train, y_train)
    
    # Vizualizace krivky uceni
    plt.plot(model.loss_curve_, label=activation.upper(), 
             color=colors[activation], linewidth=2)

plt.xlabel('Iterace (epocha)', fontsize=12)
plt.ylabel('Chyba (loss)', fontsize=12)
plt.title('Rychlost uceni pro ruzne aktivacni funkce', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\nPozorovani:")
print("- ReLU casto konverguje nejrychleji")
print("- Sigmoid muze byt pomalejsi kvuli mizejicimu gradientu")
print("- Tanh je obvykle rychlejsi nez Sigmoid")

## 11. Bonus: Softmax pro vice trid

Pro klasifikaci do vice trid se na vystupni vrstve pouziva **Softmax**, ktery prevadi vystupy na pravdepodobnosti.

In [ ]:
# Softmax pro vice trid
def softmax(x):
    """Softmax - pro vystupni vrstvu s vice tridami"""
    exp_x = np.exp(x - np.max(x))  # numericky stabilni verze
    return exp_x / exp_x.sum()

# Priklad
logits = np.array([2.0, 1.0, 0.1])  # Vystupy site pred softmax
probabilities = softmax(logits)

print("Softmax - prevod na pravdepodobnosti:")
print(f"Vstupy (logits):          {logits}")
print(f"Vystupy (pravdepodobnosti): {probabilities.round(3)}")
print(f"Soucet pravdepodobnosti:   {probabilities.sum():.4f}")

# Vizualizace
plt.figure(figsize=(10, 4))
classes = ['Trida A', 'Trida B', 'Trida C']
colors = ['#3498db', '#e74c3c', '#2ecc71']

plt.subplot(1, 2, 1)
plt.bar(classes, logits, color=colors)
plt.title('Pred Softmax (logits)', fontsize=12)
plt.ylabel('Hodnota')

plt.subplot(1, 2, 2)
plt.bar(classes, probabilities, color=colors)
plt.title('Po Softmax (pravdepodobnosti)', fontsize=12)
plt.ylabel('Pravdepodobnost')
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

## 12. Tabulka: Kdy pouzit kterou aktivacni funkci

| Funkce | Rozsah | Pouziti | Vyhody | Nevyhody |
|--------|--------|---------|--------|----------|
| **Sigmoid** | (0, 1) | Vystupni vrstva (binarni klasifikace) | Interpretace jako pravdepodobnost | Mizejici gradient |
| **Tanh** | (-1, 1) | Skryte vrstvy (RNN) | Centrovany vystup | Mizejici gradient |
| **ReLU** | [0, inf) | Skryte vrstvy (CNN, MLP) | Rychle, efektivni | Umirajici neurony |
| **Leaky ReLU** | (-inf, inf) | Skryte vrstvy | Resi umirajici neurony | O neco slozitejsi |
| **Softmax** | (0, 1), soucet=1 | Vystupni vrstva (multi-class) | Pravdepodobnostni rozdeleni | Pouze pro vystup |

## 13. Shrnuti kapitoly

### Co jsme se naucili:

1. **Aktivacni funkce** jsou klicove pro pridani nelinearity do neuronove site

2. **Sigmoid** - klasicka funkce (0-1), dobra pro vystup, ale trpi mizejicim gradientem

3. **Tanh** - podobna Sigmoidu (-1 az 1), lepsi pro skryte vrstvy

4. **ReLU** - moderni standard, rychla a efektivni, resi problem mizejiciho gradientu

5. **Problem mizejiciho gradientu** - u Sigmoid/Tanh gradient rychle konverguje k nule

### Prakticka doporuceni:
- **Skryte vrstvy:** Zacinajte s ReLU
- **Vystupni vrstva (binarni):** Sigmoid
- **Vystupni vrstva (vice trid):** Softmax
- **RNN/LSTM:** Tanh

## 14. Kviz

Otestujte sve znalosti:

In [ ]:
# Kviz
print("="*60)
print("KVIZ - Aktivacni funkce")
print("="*60)

questions = [
    {
        "question": "1. Jaky je rozsah vystupu Sigmoid funkce?",
        "options": ["a) -1 az 1", "b) 0 az 1", "c) 0 az nekonecno", "d) -nekonecno az nekonecno"],
        "answer": "b"
    },
    {
        "question": "2. Ktera aktivacni funkce je nejpouzivanejsi pro skryte vrstvy?",
        "options": ["a) Sigmoid", "b) Tanh", "c) ReLU", "d) Softmax"],
        "answer": "c"
    },
    {
        "question": "3. Co je problem mizejiciho gradientu?",
        "options": ["a) Gradient roste prilis rychle", "b) Gradient se blizi k nule", 
                   "c) Gradient je vzdy 1", "d) Gradient se nahodne meni"],
        "answer": "b"
    },
    {
        "question": "4. Jaka je maximalni hodnota derivace Sigmoid?",
        "options": ["a) 1.0", "b) 0.5", "c) 0.25", "d) 0.1"],
        "answer": "c"
    },
    {
        "question": "5. Kdy pouzijeme Softmax?",
        "options": ["a) Pro skryte vrstvy", "b) Pro binarni klasifikaci", 
                   "c) Pro klasifikaci do vice trid", "d) Pro regresi"],
        "answer": "c"
    }
]

for q in questions:
    print(f"\n{q['question']}")
    for opt in q['options']:
        print(f"   {opt}")

print("\n" + "-"*60)
print("ODPOVEDI: 1-b, 2-c, 3-b, 4-c, 5-c")
print("-"*60)

## 15. Vase vyzva

1. Zkuste pridat do site druhou skrytou vrstvu a porovnejte vysledky
2. Experimentujte s parametrem `alpha` v Leaky ReLU
3. Vytvorte vlastni dataset pomoci `make_circles()` a otestujte ruzne aktivace

**Tip:** V praxi vzdy zacinajte s ReLU a experimentujte dal pouze pokud je to potreba!